In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, precision_score

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
from pymongo import MongoClient
import ystockquote

In [4]:
from datetime import datetime
import math

# Load transcripts

In [5]:
client = MongoClient('localhost', 27017)
earnings_transcript_collection = client.python_import.earnings_transcript

In [6]:
earnings_transcript = pd.DataFrame(list(earnings_transcript_collection.find()))
earnings_transcript.drop('_id', axis=1, inplace=True)

In [7]:
earnings_transcript['publishDate_str'] = earnings_transcript.apply(
    lambda row: str(datetime.strptime(row['publishDate'], '%Y-%m-%dT%H:%M:%SZ').date()), 
    axis=1)
earnings_transcript.head(1)

,publishDate,qAndAText,rawText,tradingSymbol,url,publishDate_str
0,2016-07-27T01:01:38Z,Operator Your first question will come from Sh...,"Apple, Inc. (NASDAQ: AAPL ) Q3 2016 Earnings C...",AAPL,https://seekingalpha.com/article/3991811-apple...,2016-07-27


In [8]:
earnings_transcript.set_index(['tradingSymbol', 'publishDate_str'], inplace=True)
earnings_transcript.head(2)

publishDate  \
tradingSymbol publishDate_str                         
AAPL          2016-07-27       2016-07-27T01:01:38Z   
              2012-10-26       2012-10-26T01:33:03Z   

                                                                       qAndAText  \
tradingSymbol publishDate_str                                                      
AAPL          2016-07-27       Operator Your first question will come from Sh...   
              2012-10-26       Operator (Operator Instructions) Your first qu...   

                                                                         rawText  \
tradingSymbol publishDate_str                                                      
AAPL          2016-07-27       Apple, Inc. (NASDAQ: AAPL ) Q3 2016 Earnings C...   
              2012-10-26       Apple Inc. (NASDAQ: AAPL ) F4Q12 Earnings Call...   

                                                                             url  
tradingSymbol publishDate_str                                                     
AAPL          2016-07-27       https://seekingalpha.com/article/3991811-apple...  
              2012-10-26       https://seekingalpha.com/article/952971-apples...

In [9]:
earnings_transcript.tail(2)

publishDate  \
tradingSymbol publishDate_str                         
VRML          2013-05-15       2013-05-15T22:20:04Z   
              2013-08-14       2013-08-14T20:26:56Z   

                                                                       qAndAText  \
tradingSymbol publishDate_str                                                      
VRML          2013-05-15       Operator [Operator Instructions] Our first que...   
              2013-08-14                                                           

                                                                         rawText  \
tradingSymbol publishDate_str                                                      
VRML          2013-05-15       Vermillion (NASDAQ: VRML ) Q1 2013 Earnings Ca...   
              2013-08-14       The following audio is from an earnings confer...   

                                                                             url  
tradingSymbol publishDate_str                                                     
VRML          2013-05-15       https://seekingalpha.com/article/1438321-vermi...  
              2013-08-14       https://seekingalpha.com/article/1634312-vermi...

### Load all ticker data for transcripts

In [10]:
earnings_transcript.index.levels[0]

Index(['AAPL', 'ADBE', 'AMD', 'BLDP', 'GOOG', 'VRML'], dtype='object', name='tradingSymbol')

In [11]:
class Applyer:
    
    label = ''
    label_to_check_against = None
    
    @staticmethod
    def label_calc(row):
        
        val_to_check = 0
        if Applyer.label_to_check_against is not None:
            val_to_check = row[Applyer.label_to_check_against]
        
        if math.isnan(row[Applyer.label]) or math.isnan(val_to_check):
            return 0

        if abs(row[Applyer.label]) > val_to_check:
            if row[Applyer.label] < 0:
                return -1
            else:
                return 1
        else:
            return 0
    
    @staticmethod
    def all_label_calc(df):
        df['Std Dev'] = pd.Series(data=df['Close']).rolling(window=20,center=False).std()

        df['1day return'] = df['Close'].shift(-1) - df['Close']
        df['5day return'] = df['Close'].shift(-5) - df['Close']

        Applyer.label_to_check_against = None

        Applyer.label = '1day return'
        df['1day label'] = df.apply(Applyer.label_calc, axis=1)
        Applyer.label = '5day return'
        df['5day label'] = df.apply(Applyer.label_calc, axis=1)

        Applyer.label_to_check_against = 'Std Dev'

        Applyer.label = '1day return'
        df['1day significant label'] = df.apply(Applyer.label_calc, axis=1)
        Applyer.label = '5day return'
        df['5day significant label'] = df.apply(Applyer.label_calc, axis=1)

In [12]:
tickers = earnings_transcript.index.levels[0]
all_stocks = None
for ticker in tickers:
    stocks = ystockquote.get_historical_prices(ticker, '2000-01-01', '2017-12-31')
    print('{} ticker has {} long list'.format(ticker, len(stocks)))
    df = pd.DataFrame(stocks).transpose()
    if 'Adj Close' in df.columns:
        df['Close'] = pd.to_numeric(df['Adj Close'], errors='ignore')
    elif 'Close' in df.columns:
        df['Close'] = pd.to_numeric(df['Close'], errors='ignore')
    else:
        print('{} is not available'.format(ticker))
        continue
    df.drop(axis=1, labels=[col for col in df.columns if col not in ['Close']], inplace=True)
    indexes = pd.MultiIndex.from_product([[ticker], df.index.values.tolist()], names=['tradingSymbol', 'publishDate_str'])
    df.set_index(indexes, inplace=True)
    
    Applyer.all_label_calc(df)
    df.dropna(inplace=True)
    
    if all_stocks is None:
        all_stocks = df
    else:
        all_stocks = all_stocks.append(df)

AAPL ticker has 4339 long list
ADBE ticker has 4339 long list
AMD ticker has 4339 long list
BLDP ticker has 4339 long list
GOOG ticker has 3177 long list
VRML ticker has 4150 long list


In [13]:
len(all_stocks)

24539

In [14]:
all_stocks.sample(20)

Close   Std Dev  1day return  5day return  \
tradingSymbol publishDate_str                                                   
AMD           2003-07-03         6.710000  0.181323     0.470000     0.490000   
AAPL          2015-04-15       121.895703  1.375203    -0.586500     1.769109   
VRML          2009-11-25        19.100000  2.893484     1.900000     1.900000   
              2012-11-20         1.180000  0.052763     0.000000     0.480000   
ADBE          2002-02-05        17.274130  0.506549    -0.383206     0.766413   
AMD           2006-02-15        40.240002  1.957389     1.500000     0.149997   
              2012-07-23         4.150000  0.528274    -0.090000    -0.050000   
VRML          2017-03-16         2.200000  0.240309     0.130000    -0.060000   
AMD           2016-03-28         2.780000  0.243321     0.080000     0.050000   
BLDP          2009-09-08         1.750000  0.026926     0.020000     0.160000   
VRML          2003-03-20        53.000001  3.211394    -0.500000     0.000000   
              2009-05-07         0.030000  0.010990     1.170000     0.090000   
GOOG          2008-09-23       214.420944  9.832757     2.917095   -14.360664   
AAPL          2006-05-25         8.334566  0.439123    -0.101057    -0.345924   
ADBE          2010-02-04        32.240002  1.718616     0.529998     0.399997   
VRML          2007-08-27         8.000000  0.690919     0.600001     1.600001   
AAPL          2011-08-26        49.696452  1.511348     0.827885    -1.234702   
AMD           2005-02-18        16.900000  0.915619     0.100000     0.550001   
              2013-03-18         2.650000  0.104301     0.020000    -0.140000   
              2007-10-04        13.350000  0.324934     0.280000     0.450000   

                               1day label  5day label  1day significant label  \
tradingSymbol publishDate_str                                                   
AMD           2003-07-03                1           1                       1   
AAPL          2015-04-15               -1           1                       0   
VRML          2009-11-25                1           1                       0   
              2012-11-20                0           1                       0   
ADBE          2002-02-05               -1           1                       0   
AMD           2006-02-15                1           1                       0   
              2012-07-23               -1          -1                       0   
VRML          2017-03-16                1          -1                       0   
AMD           2016-03-28                1           1                       0   
BLDP          2009-09-08                1           1                       0   
VRML          2003-03-20               -1           0                       0   
              2009-05-07                1           1                       1   
GOOG          2008-09-23                1          -1                       0   
AAPL          2006-05-25               -1          -1                       0   
ADBE          2010-02-04                1           1                       0   
VRML          2007-08-27                1           1                       0   
AAPL          2011-08-26                1          -1                       0   
AMD           2005-02-18                1           1                       0   
              2013-03-18                1          -1                       0   
              2007-10-04                1           1                       0   

                               5day significant label  
tradingSymbol publishDate_str                          
AMD           2003-07-03                            1  
AAPL          2015-04-15                            1  
VRML          2009-11-25                            0  
              2012-11-20                            1  
ADBE          2002-02-05                            1  
AMD           2006-02-15                            0  
              2012-07-

# Merge transcripts with stock data

In [15]:
earnings_transcript = earnings_transcript.merge(all_stocks, left_index=True, right_index=True)
earnings_transcript.sample(10)

publishDate  \
tradingSymbol publishDate_str                         
GOOG          2012-04-13       2012-04-13T02:10:04Z   
AMD           2011-10-28       2011-10-28T01:10:15Z   
AAPL          2013-07-23       2013-07-23T23:38:02Z   
              2005-10-13       2005-10-13T19:00:40Z   
GOOG          2010-10-28       2010-10-28T17:05:16Z   
AAPL          2011-07-20       2011-07-20T01:40:10Z   
              2007-07-26       2007-07-26T02:58:14Z   
              2005-10-13       2005-10-13T19:06:13Z   
ADBE          2009-12-16       2009-12-16T02:51:07Z   
AMD           2012-07-20       2012-07-20T00:50:02Z   

                                                                       qAndAText  \
tradingSymbol publishDate_str                                                      
GOOG          2012-04-13       Operator [Operator Instructions] And we'll tak...   
AMD           2011-10-28       Operator [Operator Instructions] Our first que...   
AAPL          2013-07-23       Operator (Operator Instructions) Your first qu...   
              2005-10-13                                                           
GOOG          2010-10-28       Operator [Operator Instructions] It looks like...   
AAPL          2011-07-20       Operator [Operator Instructions] Your first qu...   
              2007-07-26       Operator   (Operator Instructions) First up to...   
              2005-10-13       [Rebecca Runkle of Morgan and Stanley] [Q – Re...   
ADBE          2009-12-16       Operator (Operator Instructions) We will go fi...   
AMD           2012-07-20       Operator [Operator Instructions] Our first que...   

                                                                         rawText  \
tradingSymbol publishDate_str                                                      
GOOG          2012-04-13       Google (NASDAQ: GOOG ) Q1 2012 Earnings Call A...   
AMD           2011-10-28       Advanced Micro Devices (NASDAQ: AMD ) Q3 2011 ...   
AAPL          2013-07-23       Apple Inc. (NASDAQ: AAPL ) F3Q 2013 Earnings C...   
              2005-10-13       Here's the entire text of the prepared remarks...   
GOOG          2010-10-28       Motorola (MOT) Q3 2010 Earnings Call October 2...   
AAPL          2011-07-20       Apple (NASDAQ: AAPL ) Q3 2011 Earnings Call Ju...   
              2007-07-26       Apple Inc. (NASDAQ: AAPL )   F3Q07 Earnings Ca...   
              2005-10-13       Here’s the entire text of the Q&A from Apple’s...   
ADBE          2009-12-16       Adobe Systems Incorporated (NASDAQ: ADBE ) F4Q...   
AMD           2012-07-20       Advanced Micro Devices (NASDAQ: AMD ) Q2 2012 ...   

                                                                             url  \
tradingSymbol publishDate_str                                                      
GOOG          2012-04-13       https://seekingalpha.com/article/495351-google...   
AMD           2011-10-28       https://seekingalpha.com/article/303021-advanc...   
AAPL          2013-07-23       https://seekingalpha.com/article/1566012-apple...   
              2005-10-13       https://seekingalpha.com/article/3562-apple-co...   
GOOG          2010-10-28       https://seekingalpha.com/article/233036-motoro...   
AAPL          2011-07-20       https://seekingalpha.com/article/280344-apple-...   
              2007-07-26       https://seekingalpha.com/article/42374-apple-f...   
              2005-10-13       https://seekingalpha.com/article/3563-apple-co...   
ADBE          2009-12-16       https://seekingalpha.com/article/178350-adobe-...   
AMD           2012-07-20       https://seekingalpha.com/article/734631-advanc...   

                                    Close    Std Dev  1day return  \
tradingSymbol publishDate_str                                       
GOOG          2012-04-13       311.988542   4.533853    -9.255766   
AMD           2011-10-28         5.940000   0.326076    -0.110000   
AAPL          2013-07-23        55.450012   1.565132     2.848003   
       

# Create train data and test data

In [16]:
X = earnings_transcript['rawText']
y = earnings_transcript['1day significant label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

# Tf-Idf and LogisticRegression model

### GridSearch on pipeline - it takes 20+ minutes on my machine!!!!!!!
###### http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html

In [ ]:
pipeline = Pipeline([
    ('bow', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfTransformer(norm='l2')),
    ('logreg_model', LogisticRegression())
])
#pipeline.fit(X_train, y_train)

params = dict(bow__ngram_range=[(1,1), (1,2), (1,3)],
              bow__max_df=np.arange(0.1,0.6,0.1),
              bow__min_df=[1,2],
              logreg_model__C=np.arange(0.5, 1.5, 0.3))
grid_search = GridSearchCV(pipeline, 
                           param_grid=params, 
                           verbose=3, 
                           scoring='precision_macro', 
                           n_jobs=4,
                           refit=True,
                           cv=4)
grid_search.fit(X_train, y_train)
#grid_search.fit(X, y)

Fitting 4 folds for each of 120 candidates, totalling 480 fits


[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   36.7s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  4.6min


In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

### Fitting the model

In [ ]:
pipeline = Pipeline([
    ('bow', CountVectorizer(stop_words='english', max_df=0.2, min_df=1, ngram_range=(1,1))),
    ('tfidf', TfidfTransformer(norm='l2')),
    ('bayes_model', LogisticRegression(C=0.5))
])
pipeline.fit(X_train, y_train)

In [ ]:
y_predicted = pipeline.predict(X_test)

#### Confusion matrix
By definition a confusion matrix :math:`C` is such that :math:`C_{i, j}`
is equal to the number of observations known to be in group :math:`i` but
predicted to be in group :math:`j`.

In [ ]:
cm = confusion_matrix(y_test, y_predicted)

In [ ]:
sns.heatmap(cm, cmap='magma', annot=True)

In [ ]:
cr = classification_report(y_test, y_predicted)
print(cr)